In [ ]:
import os
import cv2
from ultralytics import YOLO
import mediapipe as mp
import time

# 設定路徑
YOLO_DISPENSER_MODEL = os.path.join("/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)", "dispensor weight/best(yolov11).pt")  # 酒精機模型
VIDEO_INPUT = os.path.join("/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)", "Input Video/IMG_7129.mp4")          # 輸入影片
VIDEO_OUTPUT = os.path.join("/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)", "Output Video/processed_video.mp4")  # 輸出影片

# Mediapipe 初始化
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.3, min_tracking_confidence=0.3)
mp_draw = mp.solutions.drawing_utils

# 初始化 YOLO 模型
model_people = YOLO("yolov8n.pt")
model_dispenser = YOLO(YOLO_DISPENSER_MODEL)

# 全域變數
dispenser_roi = None
sanitized_count = 0
sanitized_ids = set()
track_state = {}
INTERSECTION_THRESHOLD = 0.1
DELAY_TIME = 3


# 影片初始化func
def initialize_video(input_path, output_path):
    """ 初始化影片讀取和寫入 """
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    return cap, out, fps, frame_width, frame_height

# 酒精偵測(yolov8)func
def detect_dispenser(frame):
    """ 偵測酒精機並返回位置 """
    results = model_dispenser(frame)
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls.item())  # 獲取類別 ID
            label = r.names[cls_id]  # 獲取class name
            confidence = box.conf[0]  # 獲取confidence
            print(f"Detected: {label}, Confidence: {confidence:.2f}")
            
            if label.lower() == "dispenser" and confidence > 0.3:  # 確保名稱匹配
                print(f"Dispenser detected at: {box.xyxy[0].tolist()}")  # 顯示座標
                return list(map(int, box.xyxy[0]))  # 返回整數座標
            
    print("No dispenser detected.")
    return None


# 計算人物與酒精機器重疊比例判斷是否"真正靠近"
def calculate_intersection_ratio(person_box, dispenser_roi):
    """ 計算人與酒精機的重疊比例 """
    def calculate_area(box):
        return max(0, box[2] - box[0]) * max(0, box[3] - box[1])

    x1 = max(person_box[0], dispenser_roi[0])
    y1 = max(person_box[1], dispenser_roi[1])
    x2 = min(person_box[2], dispenser_roi[2])
    y2 = min(person_box[3], dispenser_roi[3])
    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
    return intersection_area / min(calculate_area(person_box), calculate_area(dispenser_roi))


def process_hand_detection(frame, dispenser_roi, track_id, w, h):
    """ 使用 Mediapipe 檢測手部並確認是否進行消毒 """
    global sanitized_count
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            lower_edge_start = dispenser_roi[1] + int((dispenser_roi[3] - dispenser_roi[1]) * 0.8)
            lower_edge_end = dispenser_roi[3]
            for lm in hand_landmarks.landmark:
                hand_x = int(lm.x * w)
                hand_y = int(lm.y * h)
                if dispenser_roi[0] <= hand_x <= dispenser_roi[2] and lower_edge_start <= hand_y <= lower_edge_end:
                    if track_id not in sanitized_ids:
                        sanitized_ids.add(track_id)
                        sanitized_count += 1
                        return True
    return False


def process_tracking(frame, results_people, dispenser_roi, w, h, current_time):
    """ 處理人物追蹤邏輯 """
    for r in results_people:
        for box, track_id_tensor in zip(r.boxes, r.boxes.id):
            track_id = int(track_id_tensor.item())
            cls_id = int(box.cls.item())  # 修正此行
            label = r.names[cls_id]
            
            # 只追蹤人類
            if label != "person":
                continue
            
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            person_box = (x1, y1, x2, y2)

            # 初始化追蹤狀態
            track_state.setdefault(track_id, {'last_detected': 0, 'show_text': False})

            # 計算重疊率
            overlap_ratio = calculate_intersection_ratio(person_box, dispenser_roi)

            # 手部檢測邏輯
            if overlap_ratio > INTERSECTION_THRESHOLD:
                if (current_time - track_state[track_id]['last_detected']) > DELAY_TIME:
                    if process_hand_detection(frame, dispenser_roi, track_id, w, h):
                        track_state[track_id] = {'last_detected': current_time, 'show_text': True}

            # 顯示提示文字
            if track_state[track_id]['show_text']:
                cv2.putText(frame, "Sanitization Success!", (w // 4, h // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

            # 繪製框與 ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


# 主程式
cap, out, fps, frame_width, frame_height = initialize_video(VIDEO_INPUT, VIDEO_OUTPUT)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("處理完成")
        break

    current_time = time.time()
    h, w, _ = frame.shape

    if dispenser_roi is None:
        dispenser_roi = detect_dispenser(frame)
        continue

    results_people = model_people.track(frame, persist=True, tracker="botsort.yaml")
    process_tracking(frame, results_people, dispenser_roi, w, h, current_time)

    # 繪製消毒計數
    cv2.putText(frame, f"Sanitized Count: {sanitized_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    out.write(frame)
    cv2.imshow("Jetson Nano Sanitization Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import os
import cv2
from ultralytics import YOLO
from ultralytics.utils import LOGGER
LOGGER.setLevel("ERROR")

import mediapipe as mp
import time

# === 設定路徑 ===
YOLO_DISPENSER_MODEL = "/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/dispensor weight/best(yolov11).pt"
VIDEO_INPUT = "/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/Input Video/test2.mp4"
VIDEO_OUTPUT = "/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/Output Video/processed_video.mp4"

# === Mediapipe 初始化 ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.3, min_tracking_confidence=0.3)
mp_draw = mp.solutions.drawing_utils

# === 載入 YOLO 模型 ===
model_people = YOLO("/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/yolo11n.pt")
model_dispenser = YOLO(YOLO_DISPENSER_MODEL)

# === 全域參數 ===
dispenser_roi = None
sanitized_count = 0
sanitized_ids = set()
track_state = {}
INTERSECTION_THRESHOLD = 0.01
DELAY_TIME = 3
HAND_CHECK_INTERVAL = 5
frame_count = 0
DISPENSER_DETECT_FRAMES = 100
dispenser_detected = False

# === 初始化影片 ===
def initialize_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    return cap, out, fps, frame_width, frame_height

# === 酒精機偵測 + 畫框 ===
def detect_dispenser(frame):
    results = model_dispenser(frame)
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            confidence = box.conf[0]
            if label.lower() == "dispenser" and confidence > 0.3:
                coords = list(map(int, box.xyxy[0]))
                cv2.rectangle(frame, (coords[0], coords[1]), (coords[2], coords[3]), (255, 0, 255), 2)
                cv2.putText(frame, "Dispenser", (coords[0], coords[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                print(f"\n✅ Dispenser detected at: {coords}")
                return coords
    return None

# === 計算人物與機器的交集比例 ===
def calculate_intersection_ratio(person_box, dispenser_roi):
    def calculate_area(box):
        return max(0, box[2] - box[0]) * max(0, box[3] - box[1])
    x1 = max(person_box[0], dispenser_roi[0])
    y1 = max(person_box[1], dispenser_roi[1])
    x2 = min(person_box[2], dispenser_roi[2])
    y2 = min(person_box[3], dispenser_roi[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    return inter_area / min(calculate_area(person_box), calculate_area(dispenser_roi))

# === 手部偵測並顯示在畫面上 ===
def process_hand_detection_debug(frame, dispenser_roi, track_id, w, h, frame_count):
    global sanitized_count

    lower_y1 = dispenser_roi[1] + int((dispenser_roi[3] - dispenser_roi[1]) * 0.8)
    lower_y2 = dispenser_roi[3]
    cv2.rectangle(frame, (dispenser_roi[0], lower_y1),
                         (dispenser_roi[2], lower_y2),
                         (0, 255, 255), 2)
    overlay = frame.copy()
    cv2.rectangle(overlay, (dispenser_roi[0], lower_y1), (dispenser_roi[2], lower_y2), (0, 255, 255), -1)
    alpha = 0.3
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for lm in hand_landmarks.landmark:
                hand_x = int(lm.x * w)
                hand_y = int(lm.y * h)
                if (dispenser_roi[0] <= hand_x <= dispenser_roi[2] and
                    lower_y1 <= hand_y <= lower_y2):
                    if track_id not in sanitized_ids:
                        sanitized_ids.add(track_id)
                        sanitized_count += 1
                        print(f"✅ Person ID {track_id} triggered disinfection")
                        return True
    return False

# === 最接近人物判定函式 ===
def get_closest_person_to_dispenser(people_boxes, dispenser_roi):
    closest_id = None
    closest_distance = float('inf')
    dispenser_center = ((dispenser_roi[0] + dispenser_roi[2]) // 2,
                        (dispenser_roi[1] + dispenser_roi[3]) // 2)
    
    for track_id, person_box in people_boxes.items():
        px = (person_box[0] + person_box[2]) // 2
        py = (person_box[1] + person_box[3]) // 2
        dist = ((px - dispenser_center[0]) ** 2 + (py - dispenser_center[1]) ** 2) ** 0.5
        if dist < closest_distance:
            closest_distance = dist
            closest_id = track_id
    return closest_id

# === 人物追蹤邏輯 ===
def process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count):
    people_boxes = {}
    for r in results_people:
        if r.boxes.id is None:
            continue
        for box, track_id_tensor in zip(r.boxes, r.boxes.id):
            track_id = int(track_id_tensor.item())
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            if label != "person":
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            people_boxes[track_id] = (x1, y1, x2, y2)

    closest_id = get_closest_person_to_dispenser(people_boxes, dispenser_roi)

    for track_id, person_box in people_boxes.items():
        x1, y1, x2, y2 = person_box
        track_state.setdefault(track_id, {'last_detected': 0, 'show_text': False})

        overlap_ratio = calculate_intersection_ratio(person_box, dispenser_roi)

        if track_id == closest_id and overlap_ratio > INTERSECTION_THRESHOLD:
            if (current_time - track_state[track_id]['last_detected']) > DELAY_TIME:
                if process_hand_detection_debug(frame, dispenser_roi, track_id, w, h, frame_count):
                    track_state[track_id] = {'last_detected': current_time, 'show_text': True}

        if track_state[track_id]['show_text']:
            cv2.putText(frame, f"detected {track_id} Sanitization Success!", (w // 4, h // 2),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# === 主程式 ===
cap, out, fps, frame_width, frame_height = initialize_video(VIDEO_INPUT, VIDEO_OUTPUT)
start_time = time.time()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("處理完成")
        break

    current_time = time.time()
    h, w, _ = frame.shape

    if not dispenser_detected and frame_count < DISPENSER_DETECT_FRAMES:
        roi_candidate = detect_dispenser(frame)
        if roi_candidate:
            dispenser_roi = roi_candidate
            dispenser_detected = True
            print("Dispenser position fixed:", dispenser_roi)

    if dispenser_roi:
        results_people = model_people.track(frame, persist=True, tracker="botsort.yaml")
        process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count)

    cv2.putText(frame, f"Sanitized Count: {sanitized_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    out.write(frame)

    if frame_count % 10 == 0 and frame_count > 0:
        elapsed = time.time() - start_time
        start_time = time.time()

    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()



✅ Dispenser detected at: [0, 391, 1085, 1985]
Dispenser position fixed: [0, 391, 1085, 1985]
✅ Person ID 2 triggered disinfection
✅ Person ID 7 triggered disinfection
✅ Person ID 10 triggered disinfection
處理完成


In [19]:
import os
import cv2
from ultralytics import YOLO
from ultralytics.utils import LOGGER
LOGGER.setLevel("ERROR")

import mediapipe as mp
import time

# === 設定路徑 ===
YOLO_DISPENSER_MODEL = "/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/dispensor weight/best(yolov11).pt"
VIDEO_INPUT = "/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/Input Video/test2.mp4"
VIDEO_OUTPUT = "/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/Output Video/processed_video.mp4"

# === Mediapipe 初始化 ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.3, min_tracking_confidence=0.3)
mp_draw = mp.solutions.drawing_utils

# === 載入 YOLO 模型 ===
model_people = YOLO("/Users/yangzhelun/Desktop/alcohol_disinfection_detection(0317)/yolo11n.pt")
model_dispenser = YOLO(YOLO_DISPENSER_MODEL)

# === 全域參數 ===
dispenser_roi = None
sanitized_count = 0
sanitized_ids = set()
track_state = {}
INTERSECTION_THRESHOLD = 0.01
DELAY_TIME = 3
HAND_CHECK_INTERVAL = 5
frame_count = 0
DISPENSER_DETECT_FRAMES = 100
dispenser_detected = False

# === 初始化影片 ===
def initialize_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    return cap, out, fps, frame_width, frame_height

# === 酒精機偵測 + 畫框 ===
def detect_dispenser(frame):
    results = model_dispenser(frame)
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            confidence = box.conf[0]
            if label.lower() == "dispenser" and confidence > 0.3:
                coords = list(map(int, box.xyxy[0]))
                cv2.rectangle(frame, (coords[0], coords[1]), (coords[2], coords[3]), (255, 0, 255), 2)
                cv2.putText(frame, "Dispenser", (coords[0], coords[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                print(f"\n✅ Dispenser detected at: {coords}")
                return coords
    return None

# === 判斷手是否在出液區 ===
def is_hand_in_dispenser_zone(hand_x, hand_y, dispenser_roi):
    lower_y1 = dispenser_roi[1] + int((dispenser_roi[3] - dispenser_roi[1]) * 0.8)
    lower_y2 = dispenser_roi[3]
    return (dispenser_roi[0] <= hand_x <= dispenser_roi[2] and lower_y1 <= hand_y <= lower_y2)

# === 人物追蹤邏輯（以最接近出液區的手找最接近人物） ===
def process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count):
    global sanitized_count

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    hands_xy = []
    if results.multi_hand_landmarks:
        print(f"🖐️ Detected {len(results.multi_hand_landmarks)} hand(s)")
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            x = int(wrist.x * w)
            y = int(wrist.y * h)
            print(f"   ↳ Hand {i} wrist at ({x}, {y})")
            hands_xy.append((x, y))
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.circle(frame, (x, y), 6, (0, 0, 255), -1)

    height = dispenser_roi[3] - dispenser_roi[1]
    lower_y1 = dispenser_roi[1] + int(height * 0.8)
    lower_y2 = dispenser_roi[3] + int(height * 0.3)  # 多往下延伸 30%
    overlay = frame.copy()
    cv2.rectangle(overlay, (dispenser_roi[0], lower_y1), (dispenser_roi[2], lower_y2), (0, 255, 255), -1)
    frame[:] = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)

    people_boxes = {}
    for r in results_people:
        if r.boxes.id is None:
            continue
        for box, track_id_tensor in zip(r.boxes, r.boxes.id):
            track_id = int(track_id_tensor.item())
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            if label != "person":
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            people_boxes[track_id] = (x1, y1, x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 檢查是否有手進入出液區
    triggered_hand = None
    for hx, hy in hands_xy:
        if dispenser_roi[0] <= hx <= dispenser_roi[2] and lower_y1 <= hy <= lower_y2:
            triggered_hand = (hx, hy)
            print(f"✅ Hand entered disinfection zone at ({hx}, {hy})")
            break

    if triggered_hand and people_boxes:
        min_dist = float('inf')
        target_id = None
        for track_id, (x1, y1, x2, y2) in people_boxes.items():
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            dist = ((cx - triggered_hand[0]) ** 2 + (cy - triggered_hand[1]) ** 2) ** 0.5
            if dist < min_dist:
                min_dist = dist
                target_id = track_id
        print(f"🎯 Closest person to hand: ID {target_id} (distance={min_dist:.2f})")

        if target_id is not None:
            track_state.setdefault(target_id, {'last_detected': 0, 'show_text': False})
            if (current_time - track_state[target_id]['last_detected']) > DELAY_TIME:
                if target_id not in sanitized_ids:
                    sanitized_ids.add(target_id)
                    sanitized_count += 1
                    print(f"✅ Person ID {target_id} triggered disinfection")
                track_state[target_id] = {'last_detected': current_time, 'show_text': True}

            if track_state[target_id]['show_text']:
                cv2.putText(frame, f"detected {target_id} Sanitization Success!", (w // 4, h // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)


# === 主程式 ===
cap, out, fps, frame_width, frame_height = initialize_video(VIDEO_INPUT, VIDEO_OUTPUT)
start_time = time.time()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("處理完成")
        break

    current_time = time.time()
    h, w, _ = frame.shape

    if not dispenser_detected and frame_count < DISPENSER_DETECT_FRAMES:
        roi_candidate = detect_dispenser(frame)
        if roi_candidate:
            dispenser_roi = roi_candidate
            dispenser_detected = True
            print("Dispenser position fixed:", dispenser_roi)

    if dispenser_roi:
        results_people = model_people.track(frame, persist=True, tracker="botsort.yaml")
        process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count)

    cv2.putText(frame, f"Sanitized Count: {sanitized_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    out.write(frame)

    if frame_count % 10 == 0 and frame_count > 0:
        elapsed = time.time() - start_time
        start_time = time.time()

    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()


✅ Dispenser detected at: [0, 391, 1085, 1985]
Dispenser position fixed: [0, 391, 1085, 1985]
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (1655, 712)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (1667, 729)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (1709, 750)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (1746, 950)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (1841, 968)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (1996, 1000)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (2118, 964)
🖐️ Detected 2 hand(s)
   ↳ Hand 0 wrist at (1802, 920)
   ↳ Hand 1 wrist at (2207, 968)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (2172, 1005)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (2108, 1116)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (2003, 1165)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (2027, 1262)
🖐️ Detected 1 hand(s)
   ↳ Hand 0 wrist at (2073, 1251)
🖐️ Detected 2 hand(s)
   ↳ Hand 0 wrist at (1631, 1127)
   ↳ Hand 1 wrist at (2093, 1161)
🖐️ Detected 2 hand(s)
   ↳ Hand 0 wrist at (1824, 1161)
   ↳ H